# Deep dive into the Qumulo API python bindings

This notebook helps a user explore the Qumulo API python bindings. First, it will list out all of the supported Qunmulo API python modules and functions. **Don't forget to run <code>pip install qumulo_api</code> first.**

After listing all of the defined python functions, it demonstrates one (ore more) functions inside each of python modules. All code examples below are will only read data and configuration from your Qumulo cluster.

In [1]:
import qumulo
import os
import glob
import re
import time
import StringIO
from datetime import datetime
import dateutil.parser as date_parser
from qumulo.rest_client import RestClient

In [2]:
%%javascript
// this will prevent the large output window below from being boxed in.
IPython.OutputArea.auto_scroll_threshold = 9999;

<IPython.core.display.Javascript object>

In [3]:
# set your environment variables or fill in the variables below
API_HOSTNAME = os.environ['API_HOSTNAME'] or '{your-cluster-hostname}'
API_USER = os.environ['API_USER'] or '{api-cluster-user}'
API_PASSWORD = os.environ['API_PASSWORD'] or '{api-cluster-password}'

### Inspect all of the Qumulo python bindings and show methods

In [4]:
qumulo_lib_path = os.path.dirname(qumulo.__file__) + '/rest'

total_matches = 0
for f in glob.glob(qumulo_lib_path + '/*.py'):
    file_name = os.path.basename(f)
    if file_name == '__init__.py':
        continue
    print ""
    print "-"*80
    print "Area: %s" % (file_name, )
    c = open(f, 'r').read()
    rx_str = '@request.request[ \r\n]+def ([^(]+)\([ \r\n]*conninfo,[ \r\n]*credentials([^\)]*)(.*?)(return|yield)'
    ms = re.findall(rx_str, c, re.S|re.M)
    for m in ms:
        total_matches += 1
        func_name = m[0]
        
        # get arguments
        args = []
        arg_ms = m[1].split(',')
        for arg_m in arg_ms:
            if arg_m.strip() != "":
                args.append(re.sub('=.*', '', arg_m.strip()))

        # method
        method = "GET"
        method_m = re.search('method[ ]*=[ ]*"([A-Z]+)', m[2])
        if method_m is not None:
            method = method_m.group(1)

        # uri, currently more work for fs methods
        uri = "/"
        uri_m = re.search('uri[ ]*=.*?"([^"]+)', m[2])
        if uri_m is not None:
            uri = uri_m.group(1)
        uri_m = re.search('uri[ ]*=.*?\'([^\']+)', m[2])
        if uri_m is not None:
            uri = uri_m.group(1)

        print "   rc.%s.%s(%s)" % (file_name.replace('.py', ''), 
                                   func_name, 
                                   ', '.join(args[:4]) + (' ...' if len(args)>4 else ''))



--------------------------------------------------------------------------------
Area: ad.py
   rc.ad.list_ad()
   rc.ad.poll_ad()
   rc.ad.join_ad(domain, username, password, ou ...)
   rc.ad.leave_ad(domain, username, password)
   rc.ad.cancel_ad()
   rc.ad.uid_to_sid_get(uid)
   rc.ad.sid_to_uid_get(sid)
   rc.ad.sid_to_gid_get(sid)
   rc.ad.gid_to_sid_get(gid)
   rc.ad.sid_to_expanded_group_sids_get(sid)
   rc.ad.clear_cache_post()

--------------------------------------------------------------------------------
Area: analytics.py
   rc.analytics.time_series_get(begin_time)
   rc.analytics.iops_get(specific_type)
   rc.analytics.current_activity_get(specific_type)
   rc.analytics.capacity_history_get(interval, begin_time, end_time)
   rc.analytics.capacity_history_files_get(timestamp)

--------------------------------------------------------------------------------
Area: auth.py
   rc.auth.login(username, password)
   rc.auth.change_password(old_password, new_password)
   rc.auth.

### Create a new python REST client instance and login.

In [5]:
# Create a new reset client and login.
rc = RestClient(API_HOSTNAME, 8000)
rc.login(API_USER, API_PASSWORD)

### Show Qumulo API python client library version and Qumulo cluster software version

In [6]:
# if qumulo_api is installed via pip, this will return its version.
cmd_output = !pip show qumulo_api

# parse out results if "pip show"
pip_version = '! Unknown !'
for line in cmd_output:
    parts = line.split(':')
    if parts[0] == 'Version':
        pip_version = parts[1]

print("Qumulo API python library version: %s" % (pip_version.strip(),))

# get the Qumulo cluster software version via the API
cluster_version = rc.version.version()
print("Qumulo Cluster software version: %s" % (cluster_version['revision_id'].replace('Qumulo Core ', ''),))

# How old is the current build on the Qumulo cluster?
build_time = int(date_parser.parse(cluster_version['build_date']).strftime('%s'))
cur_time = time.time()
print("Qumulo Cluster software version is: %d days old." % ((cur_time - build_time)/(60*60*24),))

Qumulo API python library version: 2.6.2
Qumulo Cluster software version: 2.6.2
Qumulo Cluster software version is: 12 days old.


## Active Directory (ad) - list AD status

In [7]:
# Show the current status of the Cluster's AD relationship
rc.ad.list_ad()

{u'base_dn': u'',
 u'domain': u'',
 u'domain_netbios': u'',
 u'ou': u'',
 u'status': u'NOT_IN_DOMAIN',
 u'use_ad_posix_attributes': False}

## analytics - timeseries data

In [8]:
# Get the latest minute's metrics from the timeseries data endpoint.
# This data is used on the Qumulo web application's dashboard home page.
# Show the average value for the last minute for each series.
data = rc.analytics.time_series_get(begin_time=int(time.time() - 60))
for series in data:
    # skip totals since they are duplicated by the other metrics
    if 'total' in series['id']:
        continue
    print "%22s - %11s" % (series['id'],
                       round(sum(series['values']) / len(series['values']), 1))

        iops.read.rate -       229.8
       iops.write.rate -       709.2
 reclaim.deferred.rate -   1398101.3
 reclaim.snapshot.rate -         0.0
  throughput.read.rate -   1691419.6
 throughput.write.rate -  11978974.0


## Authentication and Authorization (auth) - get related identities

In [9]:
for a in rc.auth.local_username_to_all_related_identities('admin'):
    print "%(id_type)s - %(id_value)s" % a

LOCAL_GROUP - Users
NFS_UID - 0
LOCAL_USER - admin


## cluster - list nodes

In [10]:
for n in rc.cluster.list_nodes():
    print "%(node_name)s/%(id)s - %(model_number)s" % n

Product-1/1 - Q0626
Product-2/2 - Q0626
Product-3/3 - Q0626
Product-4/4 - Q0626


## dns - resolve names from ip addresses

In [11]:
for d in rc.dns.resolve(['127.0.0.1', '10.20.217.62', '192.168.0.1', '192.168.1.1', 
                       '192.168.154.1', '172.16.1.1', '10.120.246.43', '10.10.1.1']):
    print("%(ip_address)15s - %(result)10s - %(hostname)s" % d)

      127.0.0.1 -         OK - localhost
   10.20.217.62 -         OK - dyn-010-020-217-062.corp.qumulo.com
    192.168.0.1 -  NOT_FOUND - 
    192.168.1.1 -  NOT_FOUND - 
  192.168.154.1 -  NOT_FOUND - 
     172.16.1.1 -  NOT_FOUND - 
  10.120.246.43 -         OK - du68-10g.eng.qumulo.com
      10.10.1.1 -  NOT_FOUND - 


## File System (fs) - list files and read a file

In [6]:
path = '/'
dir_ent = rc.fs.read_directory(path=path)
for d in dir_ent['files']:
    if d['type'] == 'FS_FILE_TYPE_FILE':
        fw = StringIO.StringIO()
        print("Read file %(name)s which is %(size)s bytes, and print first 80 bytes" % d)
        rc.fs.read_file(fw, path = path + d['name'])
        print fw.getvalue()[:80]
        break


Read file create-lots.py which is 2576 bytes, and print first 80 bytes
import os
import sys
import random
import platform
import subprocess
from multip


## groups - list all groups in Qumulo

In [12]:
for g in rc.groups.list_groups():
    print "%(gid)6s %(id)6s %(name)16s %(sid)50s" % g


          513            Users      S-1-5-21-2200309403-3182498638-1868133574-513
  8000   1002        testgroup     S-1-5-21-2200309403-3182498638-1868133574-1002
         1003       testgroup2     S-1-5-21-2200309403-3182498638-1868133574-1003
 65534    514           Guests      S-1-5-21-2200309403-3182498638-1868133574-514


## network - show current connection counts to all nodes

In [36]:
for c in rc.network.connections():
    print "Node %2s connection count: %4s  ----  First %s: %s" % (
            c['id'], 
            len(c['connections']),
            min(len(c['connections']), 10), 
            ', '.join([d['network_address'] + '/' + d['type'].replace('CONNECTION_TYPE_', '') 
                       for d in c['connections']]))

Node  1 connection count:    1  ----  First 1: 10.120.247.95/NFS
Node  2 connection count:    0  ----  First 0: 
Node  3 connection count:    1  ----  First 1: 10.120.247.91/NFS
Node  4 connection count:    2  ----  First 2: 10.120.247.92/NFS, 10.120.247.92/NFS


## nfs - list shares

In [39]:
for share in rc.nfs.nfs_list_shares():
    print "%(export_path)s -> %(fs_path)s - %(description)s" % share

/ -> / - 
/users/hazel -> /users/hazel - NFS share user: hazel
/users/tommy -> /users/tommy - NFS share user: tommy
/users/keith -> /users/keith - NFS share user: keith
/users/sarah -> /users/sarah - NFS share user: sarah
/users/stephanie -> /users/stephanie - NFS share user: stephanie
/users/mashton -> /users/mashton - 


## node_state - get current node state

In [42]:
print rc.node_state.get_node_state()
# (only prints the state of the node the rest client is currently connected to)

{u'state': u'ACTIVE', u'node_id': 1, u'cluster_id': u'dd7a29af-c798-4cf5-ab74-00e0f4124afe'}


## quota - list current quotas and capacity used for each

In [51]:
for qd in rc.quota.get_all_quotas_with_status():
    for q in qd['quotas']:
        print "%(path)s - id: %(id)s - %(capacity_usage)s bytes used of %(limit)s" % q

/justin/test1/ - id: 1381143386 - 4096 bytes used of 10000000000
/justin/demo3/ - id: 1388146039 - 4096 bytes used of 10000000000
/justin/demo1/ - id: 1405146087 - 4096 bytes used of 10000000000
/justin/demo2/ - id: 1406145934 - 4096 bytes used of 10000000000
/justin/test2/ - id: 1435143613 - 4096 bytes used of 10000000000


## smb - list shares

In [53]:
for share in rc.smb.smb_list_shares():
    print "%(share_name)s -> %(fs_path)s - %(description)s" % share

Files -> / - 
testmatt -> /testmatt - 
testmatt2 -> /testmatt2 - 
testmatt3 -> /testmatt3 - 
testmatt4 -> /testmatt4 - 
testmatt5 -> /testmatt5 - 
testwill -> /testwill - 
users -> /users - 
miketest -> /miketest - 
Music -> /Music/ - Test for demo
replication -> /replication - Dirs for SE replication demos


## snapshots - list directory-level snapshots

In [57]:
for snap in rc.snapshot.list_snapshots()['entries']:
    print("%(name)s - %(source_path)s - %(directory_name)s - %(timestamp)s" % snap)

TommyRootSnapshot - / - 82062_TommyRootSnapshot - 2017-03-10T18:45:36.259121052Z


## support - check support/monitoring status

In [59]:
rc.support.get_config()

{u'enabled': True,
 u'mq_host': u'missionq.qumulo.com',
 u'mq_port': 443,
 u'period': 60,
 u's3_proxy_disable_https': False,
 u's3_proxy_host': u'monitor.qumulo.com',
 u's3_proxy_port': 443,
 u'vpn_enabled': False,
 u'vpn_host': u'ep1.qumulo.com'}

## time_config - get current time status

In [62]:
rc.time_config.get_time_status()

{u'config': {u'ntp_servers': [u'0.qumulo.pool.ntp.org',
   u'1.qumulo.pool.ntp.org'],
  u'use_ad_for_primary': True},
 u'time': u'2017-04-03T04:51:20.490766807Z'}

## users - list users

In [68]:
for user in rc.users.list_users():
    print("%(name)10s - %(uid)5s - %(id)5s - %(primary_group)4s - %(sid)s" % user)

    qumulo -       -  1000 -  513 - S-1-5-21-2200309403-3182498638-1868133574-1000
     guest - 65534 -   501 -  514 - S-1-5-21-2200309403-3182498638-1868133574-501
  testmatt -  2001 -  1001 -  513 - S-1-5-21-2200309403-3182498638-1868133574-1001
      mike -       -  1004 -  513 - S-1-5-21-2200309403-3182498638-1868133574-1004
        SE -       -  1005 -  513 - S-1-5-21-2200309403-3182498638-1868133574-1005
     admin -     0 -   500 -  513 - S-1-5-21-2200309403-3182498638-1868133574-500
